In [ ]:
using Random
using CoordinateTransformations, Rotations
using StaticArrays
using Gadfly
using Statistics
using NearestNeighbors
using LinearAlgebra

Let's create a random 2D data set to represent points on a plane.

In [ ]:
r = MersenneTwister(0xBADFEED)
npts = 100
xy = SVector.(zip(rand(r, npts), rand(r, npts)))

Now we are going to muddle this dataset by translating, rotating, adding a small random component and shuffling the data.

In [ ]:
xyp = AffineMap(Angle2d{Float64}(deg2rad(1.2)), [0.1, -0.1]).(xy) .+ 0.02 * SVector.(collect(zip(0.5 .- rand(r, npts), 0.5 .- rand(r, npts))))
ro = shuffle(1:100)  # Use this to reorder the points
xyps = xyp[ro]

Let's take a look at the result (red->original points, blue->muddled points)

In [ ]:
set_default_plot_size(6inch, 10inch)
vstack(plot(
        layer(x = map(p -> p[1], xy), y = map(p -> p[2], xy), Geom.point, Gadfly.Theme(default_color = "red")),
        layer(x = map(p -> p[1], xyps), y = map(p -> p[2], xyps), Geom.point, Gadfly.Theme(default_color = "blue")),
    ),
    plot(
        layer(x = map(p -> p[1], xy[ro]), y = map(p -> p[2], xy[ro]), color = 1:100, Geom.point, Gadfly.Theme(default_color = "red")),
        layer(x = map(p -> p[1], xyps), y = map(p -> p[2], xyps), color = 1:100, Geom.point, Gadfly.Theme(default_color = "blue")),
    ))

We will start to restore the muddled points by centering the muddled points on the original points.
We calculate the center of mass for each set of points and translate the muddled points by the difference in center of mass.

In [ ]:
centerofmass(pts) = [mean(p -> p[1], pts), mean(p -> p[2], pts)]
xypst = Translation(centerofmass(xy) - centerofmass(xyps)).(xyps)

This brings the points into better registration but there remains the shuffle and the rotation to deal with.

In [ ]:
set_default_plot_size(6inch, 10inch)
vstack(
    plot(
        layer(x = map(p -> p[1], xy), y = map(p -> p[2], xy), Geom.point, Gadfly.Theme(default_color = "red")),
        layer(x = map(p -> p[1], xypst), y = map(p -> p[2], xypst), Geom.point, Gadfly.Theme(default_color = "blue")),
        Coord.cartesian(xmin = 0.0, xmax = 1.0, ymin = 0.0, ymax = 1.0)
    ),
    plot(
        layer(x = map(p -> p[1], xy[ro]), y = map(p -> p[2], xy[ro]), color = 1:100, Geom.point, Gadfly.Theme(default_color = "red")),
        layer(x = map(p -> p[1], xypst), y = map(p -> p[2], xypst), color = 1:100, Geom.point, Gadfly.Theme(default_color = "blue")),
        Coord.cartesian(xmin = 0.0, xmax = 1.0, ymin = 0.0, ymax = 1.0)
    )
)

We will use a KDTree nearest-neighbor algorithm to identify which point in `xy` is closest to each point in the muddled data.
Find the closest neighbor for each particle in `xypt` to `xy`.  Then reorder `xypt` to align each point with its closest matching point in `xy`.

In [ ]:
tr = KDTree(xypst)
i_knn = @time collect(Iterators.flatten(knn(tr, xy, 1)[1]))
xypstr = xypst[i_knn]
@show xy[1:10]
@show xypstr[1:10];

Now the muddled data and the original data are back in roughtly the same order.

In [ ]:
set_default_plot_size(6inch, 4inch)
plot(
    layer(x = map(p -> p[1], xy), y = map(p -> p[2], xy), color = 1:100, Geom.point, Gadfly.Theme(default_color = "red")),
    layer(x = map(p -> p[1], xypstr), y = map(p -> p[2], xypstr), color = 1:100, Geom.point, Gadfly.Theme(default_color = "blue")),
    Coord.cartesian(xmin = 0.0, xmax = 1.0, ymin = 0.0, ymax = 1.0)
)

Now we apply the solution to the "orthogonal procrustes problem" to determine the rotation that will bring the muddled data back in alignment with the original data.

In [ ]:
m2(vs) = transpose(reshape(collect(Iterators.flatten(vs)), 2, 100))
f = svd(m2(xy) * transpose(m2(xypstr)))
Ω = f.U * f.Vt
res = Ω * m2(xypstr)

Which brings the points back into alignment...  Usually, it is necessary to iterate the nearest neighbor and orthogonal crustes steps to produce the best solution. 

In [ ]:
plot(
    layer(x = map(p -> p[1], xy), y = map(p -> p[2], xy), Geom.point, Gadfly.Theme(default_color = "red")),
    layer(x = res[:, 1], y = res[:, 2], Geom.point, Gadfly.Theme(default_color = "blue")),
    Coord.cartesian(xmin = 0.0, xmax = 1.0, ymin = 0.0, ymax = 1.0)
)

In [ ]:
plot(x = map(p -> p[1], xy) - res[:, 1], y = map(p -> p[2], xy) - res[:, 2], Geom.point, Gadfly.Theme(default_color = "red"))

Problems with this algorithm.
1) Isn't likely to converge to the correct answer when the rotation is large.
2) Works best when the point sets have a one-to-one point correspondence (or nearly one-to-one)

In [ ]:
centerofmass(pts::AbstractVector{T}) where { T <: AbstractVector{<:AbstractFloat} } = #
    [mean(p -> p[i], pts) for i in 1:length(pts[1])]

"""
    icp_iteration(pts1::AbstractVector{<:AbstractVector{2, T}}, pts2::AbstractVector{<:AbstractVector{2, T}}; orthonormal=false) where { T<: AbstractFloat }

Perform a single iteration of the iterative closest point algorithm using the orthogonal Procrustes algorithm.

Returns the orthogonal matrix Ω such that Ω⋅pts2 ≈ pts1.  If `orthonormal=true` then Ω is converted to the orthonormal rotation matrix. 
"""
function icp_iteration(pts1::AbstractVector{T}, pts2::AbstractVector{T}; orthonormal = false) where { T <: AbstractVector{<:AbstractFloat}}
    # Center both pts1 and pts2 on the origin
    com1, com2 = centerofmass(pts1), centerofmass(pts2)
    pts1c, pts2c = Translation(-com1).(pts1), Translation(-com2).(pts2)
    # Find the nearest neighbors for pts2c in pts1c
    tr = KDTree(pts2c)
    i_knn = collect(Iterators.flatten(knn(tr, pts1c, 1)[1]))
    # Reorder pts2c to best correspond to pts1's order
    pts2cr = pts2c[i_knn]
    # Turn Vector{Vector{<:AbstractFloat}} into Matrix{<:AbstractFloat}
    m2(vs) = transpose(reshape(collect(Iterators.flatten(vs)), length(vs[1]), length(vs)))
    # Solve the orthogonal Procrustes problem
    f = svd(m2(pts1c) * transpose(m2(pts2cr)))
    Ω = f.U * f.Vt
    if orthonormal
        # Convert the orthogonal matrix into an orthonormal rotation matrix
        Σp = ones(eltype(f.S), length(f.S))
        Σp[findmin(f.S)[2]] = det(Ω)
        Ω = f.U * Diagonal(Σp) * f.Vt
    end
    return Translation(com1).(Ω * pts2cr)
end

In [ ]:
res = icp_iteration(xy, xyps, orthonormal = false)
@show res[1:5]
@show xy[1:5];

In [ ]:
plot(x = map(p -> p[1], xy) - map(p -> p[1], res), y = map(p -> p[2], xy) - map(p -> p[2], res), Geom.point, Gadfly.Theme(default_color = "red"))

In [ ]:
res2 = icp_iteration(xy, res, orthonormal = false)
plot(x = map(p -> p[1], xy) - map(p -> p[1], res2), y = map(p -> p[2], xy) - map(p -> p[2], res2), Geom.point, Gadfly.Theme(default_color = "red"))

In [ ]:
function icp(pts1::AbstractVector{T}, pts2::AbstractVector{T}; maxiter=10, tol=0.9) where { T <: AbstractVector{<:AbstractFloat}}
    function err(pts1, pts2)
        # Associate each point in pts2 with the closest point in pts1 
        pts2r = pts2[collect(Iterators.flatten(knn(KDTree(pts2), pts1, 1)[1]))]
        # Compute the sum Euclidean distance
        sum(norm(p1 - p2) for (p1, p2) in zip(pts1, pts2r))
    end
    preverror, next = err(pts1, pts2), pts2
    for _ in 1:maxiter
        next = icp_iteration(pts1, next)
        nexterror = err(pts1, next)
        if nexterror > tol*preverror
            return next
        end
        preverror = nexterror
    end
    @warn "Not converging after $maxiter steps in icp(...)."
    return next
end

In [ ]:
res3 = @time icp(xy, xyps, tol=0.999)
plot(x = map(p -> p[1], xy) - map(p -> p[1], res3), y = map(p -> p[2], xy) - map(p -> p[2], res3), Geom.point, Gadfly.Theme(default_color = "red"))
